In [13]:
import numpy as np
import pandas as pd
import json
import re
import nltk
nltk.download('stopwords') #make sure list up to date

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\magar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
#read in scraped data
with open('charts_and_lyrics.json') as json_data:
    inputData = json.load(json_data)

In [136]:
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmaCounts(lyrics, chartName):    
    #make sure lyrics were found for this song
    #lyrics will be False if none were found
    if lyrics:
        #strip "[Chorus]", "[Verse 1]" etc tags
        lyrics = re.sub(r'\[.*\]', '', lyrics)
        
        #strip any punctuation
        lyrics = re.sub(r'\W',' ',lyrics)
        
        #replace any multiple consecutive spaces with just one space
        lyrics = re.sub(r'\s+',' ',lyrics)
        
        words = nltk.word_tokenize(lyrics.lower())

        #add words to word count lists
        for word in words:
            if word not in stopwords:
                lemma = lemmatizer.lemmatize(word)

                #add words to master list
                if lemma not in allChartsLemmaCounts.keys():
                    allChartsLemmaCounts[lemma] = {'count': 1, 'sourceWords': [word]}
                else:
                    allChartsLemmaCounts[lemma]['count'] = dic[lemma]['count'] + 1
                    if word not in allChartsLemmaCounts[lemma]['sourceWords']:
                        allChartsLemmaCounts[lemma]['sourceWords'].append(word)

        return

In [137]:
allChartsLemmaCounts = {}

for chart in inputData:
    for song in chart['entries']:
        lemmaCounts(song['lyrics'], chart['name'])

In [138]:
allChartsLemmaCountsDf = pd.DataFrame.from_dict(allChartsLemmaCounts, orient='index')

#currently the index of the df is the words and the counts are in a column called 0
allChartsLemmaCountsDf.sort_values(by='count', ascending=False, inplace=True)

#create an actual index and move the words into a column
allChartsLemmaCountsDf.reset_index(inplace=True)
allChartsLemmaCountsDf.columns = ['word', 'count', 'sourceWords']

allChartsLemmaCountsDf

,word,count,sourceWords
0,yeah,1859,[yeah]
1,like,1720,"[like, likes]"
2,know,1646,"[know, knows]"
3,oh,1634,[oh]
4,got,1466,[got]
5,na,1380,[na]
6,love,1299,"[love, loves]"
7,get,974,"[get, gets]"
8,let,833,"[let, lets]"
9,go,795,"[go, goes]"


In [121]:
max(allChartsLemmaCountsDf.sourceWords.apply(len))

2